In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

In [14]:
teams_half = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/TeamsHalf.csv')
batting_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/BattingPost.csv')
fielding_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/FieldingPost.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
salaries = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/Salaries.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
series_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/SeriesPost.csv')

In [15]:
teams_half.head()

,yearID,lgID,teamID,Half,divID,DivWin,Rank,G,W,L
0,1981,NL,ATL,1,W,N,4,54,25,29
1,1981,NL,ATL,2,W,N,5,52,25,27
2,1981,AL,BAL,1,E,N,2,54,31,23
3,1981,AL,BAL,2,E,N,4,51,28,23
4,1981,AL,BOS,1,E,N,5,56,30,26


In [18]:
teams_half.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 10 columns):
yearID    52 non-null int64
lgID      52 non-null object
teamID    52 non-null object
Half      52 non-null int64
divID     52 non-null object
DivWin    52 non-null object
Rank      52 non-null int64
G         52 non-null int64
W         52 non-null int64
L         52 non-null int64
dtypes: int64(6), object(4)
memory usage: 4.1+ KB


In [19]:
teams_half.describe()

,yearID,Half,Rank,G,W,L
count,52.0,52.000000,52.000000,52.000000,52.000000,52.000000
mean,1981.0,1.500000,3.692308,53.423077,26.711538,26.711538
std,0.0,0.504878,1.842190,2.872084,5.333345,5.184201
min,1981.0,1.000000,1.000000,48.000000,15.000000,20.000000
25%,1981.0,1.000000,2.000000,52.000000,23.000000,23.000000
50%,1981.0,1.500000,4.000000,53.000000,27.000000,26.000000
75%,1981.0,2.000000,5.000000,56.000000,31.000000,29.000000
max,1981.0,2.000000,7.000000,60.000000,37.000000,42.000000


In [22]:
teams_half.columns

Index(['yearID', 'lgID', 'teamID', 'Half', 'divID', 'DivWin', 'Rank', 'G', 'W',
       'L'],
      dtype='object')

In [25]:
batting_p.head()

,yearID,round,playerID,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,1884,WS,becanbu01,NY4,AA,1,2,0,1,0,...,0,0,NaN,0,0,0,NaN,NaN,NaN,NaN
1,1884,WS,bradyst01,NY4,AA,3,10,1,0,0,...,0,0,NaN,0,1,0,NaN,NaN,NaN,NaN
2,1884,WS,carrocl01,PRO,NL,3,10,2,1,0,...,1,0,NaN,1,1,0,NaN,NaN,NaN,NaN
3,1884,WS,dennyje01,PRO,NL,3,9,3,4,0,...,2,0,NaN,0,3,0,NaN,NaN,NaN,NaN
4,1884,WS,esterdu01,NY4,AA,3,10,0,3,1,...,0,1,NaN,0,3,0,NaN,NaN,NaN,NaN


In [26]:
batting_p.info

<bound method DataFrame.info of        yearID  round   playerID teamID lgID  G  AB  R   H  2B  ...   RBI  SB  \
0        1884     WS  becanbu01    NY4   AA  1   2  0   1   0  ...     0   0   
1        1884     WS  bradyst01    NY4   AA  3  10  1   0   0  ...     0   0   
2        1884     WS  carrocl01    PRO   NL  3  10  2   1   0  ...     1   0   
3        1884     WS  dennyje01    PRO   NL  3   9  3   4   0  ...     2   0   
4        1884     WS  esterdu01    NY4   AA  3  10  0   3   1  ...     0   1   
5        1884     WS  farreja02    PRO   NL  3   9  3   4   2  ...     0   1   
6        1884     WS  forstto01    NY4   AA  1   3  0   0   0  ...     0   0   
7        1884     WS  gilliba01    PRO   NL  3   9  3   4   2  ...     2   0   
8        1884     WS  hinespa01    PRO   NL  3   8  5   2   0  ...     1   2   
9        1884     WS  irwinar01    PRO   NL  3   9  3   2   0  ...     2   0   
10       1884     WS  keefeti01    NY4   AA  2   5  0   1   0  ...     0   0   
11      

In [29]:
batting_p.describe

<bound method NDFrame.describe of        yearID  round   playerID teamID lgID  G  AB  R   H  2B  ...   RBI  SB  \
0        1884     WS  becanbu01    NY4   AA  1   2  0   1   0  ...     0   0   
1        1884     WS  bradyst01    NY4   AA  3  10  1   0   0  ...     0   0   
2        1884     WS  carrocl01    PRO   NL  3  10  2   1   0  ...     1   0   
3        1884     WS  dennyje01    PRO   NL  3   9  3   4   0  ...     2   0   
4        1884     WS  esterdu01    NY4   AA  3  10  0   3   1  ...     0   1   
5        1884     WS  farreja02    PRO   NL  3   9  3   4   2  ...     0   1   
6        1884     WS  forstto01    NY4   AA  1   3  0   0   0  ...     0   0   
7        1884     WS  gilliba01    PRO   NL  3   9  3   4   2  ...     2   0   
8        1884     WS  hinespa01    PRO   NL  3   8  5   2   0  ...     1   2   
9        1884     WS  irwinar01    PRO   NL  3   9  3   2   0  ...     2   0   
10       1884     WS  keefeti01    NY4   AA  2   5  0   1   0  ...     0   0   
11    

In [39]:
## Can Drop a good majority of values in CS HBP SH SF AND GIDP 
batting_p.isnull().sum()

yearID        0
round         0
playerID      0
teamID        0
lgID          0
G             0
AB            0
R             0
H             0
2B            0
3B            0
HR            0
RBI           0
SB            0
CS          201
BB            0
SO            0
IBB           0
HBP         201
SH          201
SF          201
GIDP        201
dtype: int64